In [38]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import interact
import ipywidgets as widgets
import folium
import warnings
warnings.simplefilter('ignore')
from IPython.display import display, HTML
from pandas import datetime
import io
import re

In [39]:
 %%HTML
<p><img src="bannr.png" align="center"></img></p>

In [40]:
%%HTML

<div style = "text-align: justify;">
<div style = "Trebuchet MS;font-size:20px;color:#999999;text-align:center;" ><h2>Interactive Analysis Dashboard</h2></div><br>
<div style = "Trebuchet MS;font-size:20px;color:#999999;line-height: 1.6;text-align: justify;" >Coronaviruses are a large family of viruses that may cause respiratory illnesses in humans ranging from common colds to more severe conditions such as Severe Acute Respiratory Syndrome (SARS) and Middle Eastern Respiratory Syndrome (MERS). Novel coronavirus is a new, previously unidentified strain of coronavirus. The coronavirus involved in the current outbreak has been named SARS-CoV by the World Health Organization (WHO).The disease it causes has been named “coronavirus disease 2019” or “COVID-19”.</div>
</div>

In [41]:
# Loading the online state wise data from csv files 

statewise_df = pd.read_csv('https://api.covid19india.org/csv/latest/state_wise.csv')
latlong_df = pd.read_csv('latlong.csv')

In [42]:
mydateparser = lambda x: pd.datetime.strptime(str(x), "%d/%m/%Y")

In [43]:
district_datewise_df1 = pd.read_csv('https://api.covid19india.org/csv/latest/raw_data1.csv',parse_dates= True, infer_datetime_format=True,date_parser=mydateparser)
district_datewise_df1 = district_datewise_df1[['Detected State','Detected District','Detected City','Date Announced','Num cases','Current Status']]
district_datewise_df1.columns = map(str.lower, district_datewise_df1.columns)
#print(district_datewise_df1.shape)

district_datewise_df2 = pd.read_csv('https://api.covid19india.org/csv/latest/raw_data2.csv',parse_dates= True, infer_datetime_format=True,date_parser=mydateparser)
district_datewise_df2 = district_datewise_df2[['Detected State','Detected District','Detected City','Date Announced','Num cases','Current Status']]
district_datewise_df2.columns = map(str.lower, district_datewise_df2.columns)
#print(district_datewise_df2.shape)

district_datewise_df3 = pd.read_csv('https://api.covid19india.org/csv/latest/raw_data3.csv',parse_dates= True, infer_datetime_format=True,date_parser=mydateparser)
district_datewise_df3 = district_datewise_df3[['Detected State','Detected District','Detected City','Date Announced','Num Cases','Current Status']]
district_datewise_df3.columns = map(str.lower, district_datewise_df3.columns)
#print(district_datewise_df3.shape)

district_datewise_df4 = pd.read_csv('https://api.covid19india.org/csv/latest/raw_data4.csv',parse_dates= True, infer_datetime_format=True,date_parser=mydateparser)
district_datewise_df4 = district_datewise_df4[['Detected State','Detected District','Detected City','Date Announced','Num Cases','Current Status']]
district_datewise_df4.columns = map(str.lower, district_datewise_df4.columns)
#print(district_datewise_df4.shape) 

# district_datewise_df5 = pd.read_csv('https://api.covid19india.org/csv/latest/raw_data5.csv',parse_dates= True, infer_datetime_format=True,date_parser=mydateparser)
# district_datewise_df5 = district_datewise_df4[['Detected State','Detected District','Detected City','Date Announced','Num Cases','Current Status']]
# district_datewise_df5.columns = map(str.lower, district_datewise_df4.columns)
# print(district_datewise_df5.shape) 



district_datewise_df = pd.concat([district_datewise_df1,district_datewise_df2,district_datewise_df3,district_datewise_df4], axis=0)
print(district_datewise_df.shape)


# for url_no in range(2,5):
    
#     url = 'https://api.covid19india.org/csv/latest/raw_data{n}.csv'
#     url_no = str(url_no)
#     print(url_no)
#     new_url = url.replace('{n}', url_no)
#     print(new_url)
   
#     read_csv =  pd.read_csv(new_url)
#     district_datewise_df = pd.concat([district_datewise_df1,read_csv], axis=0)
#    print(district_datewise_df.shape)



(56423, 6)


In [44]:
# Droping the the first row total
statewise_df.drop(statewise_df.index[[0]],axis=0, inplace = True)

In [45]:

indicator_df  = statewise_df.copy()
indicator_df.columns = map(str.lower, indicator_df.columns)

In [46]:
indicator_df = indicator_df.drop(['state_code','last_updated_time','state_notes'], axis = 1)


In [47]:
# creating dataframe for the folium map merging lat long dataframe

statewise_folium_df = statewise_df.copy()
statewise_folium_df = statewise_folium_df.replace('State Unassigned','India')
statewise_folium_df = statewise_folium_df[['State','Confirmed','Recovered','Deaths','Active']]
statewise_folium_df.sort_values(by=['State','Confirmed'] , inplace=True, ascending=True)
statewise_folium_df.reset_index(inplace = True, drop = True)
lat_long_merged_df = pd.concat([statewise_folium_df, latlong_df], axis = 1)
#lat_long_merged_df

In [48]:
# droping Extra column from the lat long df

lat_long_merged_df.drop(['State_lat_long'], inplace = True, axis=1)


In [49]:
# coverting all the coloumss into lower cases using map function 

statewise_df.columns = map(str.lower, statewise_df.columns)


In [50]:
# changing province/state to state and state/region to state

statewise_df = statewise_df.rename(columns={'last_updated_time': 'last_update'})


In [51]:
# Changing some column name for the readibility purpose.

district_datewise_df = district_datewise_df.rename(columns={'detected state': 'state','detected district':'district','detected city':'city','date announced':'date','num cases':'count','Current Status':'status'})


In [52]:
# Fetching the last_update time from the statewise_df

last_update = statewise_df['last_update'].values[0]
display(HTML('<div style = "font-family:Trebuchet MS;font-size:12px;color:#999999;align:center"><h1>Last Updated</h1></div>'))


def change_date_format(dt):
        return re.sub(r'(\d{4})-(\d{1,2})-(\d{1,2})', '\\3-\\2-\\1', dt)
dt1 = "2026-01-02"
print("\033[1;34;143m", change_date_format(last_update), "\n")


 03/06/2020 20:46:04 



In [53]:
total_indicators = indicator_df.iloc[0,:]

confirmed = total_indicators.iloc[1]
recovered = total_indicators.iloc[2]
deaths = total_indicators.iloc[3]
active = total_indicators.iloc[4]

delta_confirmed = total_indicators.iloc[5]
delta_recovered = total_indicators.iloc[6]
delta_deaths = total_indicators.iloc[7]

delta_active = active-(+delta_confirmed+delta_recovered+delta_deaths)


# largest = indicator_df['active'].nlargest(2)
# first_largest  = largest.ix[0]
# second_largest = largest.ix[1]
# delta_active = first_largest-second_largest
delta_active

display(HTML('<br><br><div style = "font-family:Trebuchet MS;font-size:30px;color:#999999;">Daily Indicator</div><br>'))


fig = go.Figure()

fig.add_trace(
        go.Indicator(
            mode = "number+delta",
            title = {"text": "<span style='font-family:Trebuchet MS;font-size:30px;color:#0000FF'><b>Confirmed</b></span>",'align':'center'},
            value = confirmed,
            align="center",
            delta = {'reference':confirmed-delta_confirmed,'increasing': {'color': "#0000FF"}, 'relative': False, 'position' : "top",'valueformat' :".0f"},
            domain = {'row': 0, 'column': 0}
            
        )
)


fig.add_trace(go.Indicator(
    
    mode = "number+delta",
    title = {"text": "<span style=';font-family:Trebuchet MS;font-size:30px;color:#7CFC00'><b>Recovered</b></span>",'align':'center'},
    value = recovered,
    align="center",
    delta = {'reference':recovered-delta_recovered,'increasing': {'color': "#7CFC00"}, 'relative': False,'position' : "top",'valueformat' :".0f"},
    domain = {'row': 1, 'column': 0},

))



fig.add_trace(go.Indicator(
    mode = "number+delta",
    title = {"text": "<span style=';font-family:Trebuchet MS;font-size:30px;color:#ff33cc'><b>Active</b></span>",'align':'center'},
    value = active,
    align="center",
    delta = {'reference': delta_active,'increasing': {'color': "#ff33cc"}, 'relative': False,'position' : "top",'valueformat' :".0f"},
    domain = {'row': 0, 'column': 1}
))
  


fig.add_trace(go.Indicator(
    mode = "number+delta",
    title = {"text": "<span style=';font-family:Trebuchet MS;font-size:30px;color:#FF0000'><b>Death</b></span>",'align':'center'},
    value = deaths,
    align="center",
    delta = {'reference': deaths-delta_deaths, 'increasing': {'color': "#FF0000"}, 'relative': False,'position' : "top",'valueformat' :".0f"},
    domain = {'row': 1, 'column': 1}
    
))
    

fig.update_layout(
    grid = {'rows': 2, 'columns': 2, 'pattern': "coupled"},
    template = {'data' : {'indicator': [{
        'delta' : {'reference': 90}}]
                         }})




In [54]:
district_table_df = pd.DataFrame((district_datewise_df.groupby( [ "state","district"] ).sum()).reset_index())
#district_table_df


# Set CSS properties for th elements in dataframe
th_props = [
  ('font-size', '20px'),
  ('text-align', 'center'),
  ('font-weight', 'bold'),
  ('color', '#999999'),
  ('background-color', '#f7f7f9')
  ]

# Set table styles
styles = [
  dict(selector="th", props=th_props)
  ]


# Fetching the number of desired rows using interaction.

fig = go.FigureWidget( layout=go.Layout() )

def highlight_col(x):
    
                    state   = 'background-color: #ffffcc'
                    district = 'background-color: #99ebff'
                    count     = 'background-color: #ffc2b3'
                    
                    

                    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
                    df1.iloc[:, 0] = state
                    df1.iloc[:, 1] = district
                    df1.iloc[:, 2] = count
                   
                   


                    return df1

def show_latest_cases(Number):
    n = int(Number)
   
    pd.options.display.float_format = '{:.2f}'.format
    s = district_table_df.sort_values('count', ascending= False).head(n).style.apply(highlight_col, axis=None) 
    s = s.hide_index()
    s.set_precision(20)
    s.set_table_styles(styles)
    s.set_properties(**{'border-style': 'solid',
                        'border-color': '#e0e0eb',
                        'border-width':'0.015px',
                        'color': 'black',
                        'font-family':'Trebuchet MS',
                        'font-weight':'bold',
                        'text-align':'center',
                        'width':'200px'
                           })
    #s = s.set_caption('<h3>Top Worst Hit States</h3><br>')
    pd.options.display.float_format = '{:.2f}'.format
    pd.option_context('display.colheader_justify','center')
    
    return s;

#display(HTML('<br><br><div style = "font-family:Trebuchet MS;font-size:24px;color:#999999;">Enter the Number to View Daily Record</div><br>'))
interact(show_latest_cases, Number='10');


interactive(children=(Text(value='10', description='Number'), Output()), _dom_classes=('widget-interact',))

In [55]:
# Set CSS properties for th elements in dataframe
th_props = [
  ('font-size', '20px'),
  ('text-align', 'center'),
  ('font-weight', 'bold'),
  ('color', '#999999'),
  ('background-color', '#f7f7f9')
  ]

# Set table styles
styles = [
  dict(selector="th", props=th_props)
  ]






# Fetching the number of desired rows using interaction.

fig = go.FigureWidget( layout=go.Layout() )

def highlight_col(x):
    
                    state   = 'background-color: #ffffcc'
                    confirmed = 'background-color: #99ebff'
                    death     = 'background-color: #ffc2b3'
                    recovered = 'background-color: #99ff99'
                    active    = 'background-color: #ffb3ff'
                    

                    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
                    df1.iloc[:, 0] = state
                    df1.iloc[:, 1] = confirmed
                    df1.iloc[:, 2] = death
                    df1.iloc[:, 3] = recovered
                    df1.iloc[:, 4] = active
                   


                    return df1

def show_latest_cases(Number):
    n = int(Number)
    if 'delta_confirmed' in statewise_df:
        statewise_df.drop(['last_update','state_code','delta_confirmed','delta_recovered','delta_recovered','state_notes','delta_deaths'], axis =1, inplace =True)

    pd.options.display.float_format = '{:.2f}'.format
    s = statewise_df.sort_values('confirmed', ascending= False).head(n).style.apply(highlight_col, axis=None) 
    s = s.hide_index()
    s.set_precision(20)
    s.set_table_styles(styles)
    s.set_properties(**{'border-style': 'solid',
                        'border-color': '#e0e0eb',
                        'border-width':'0.015px',
                        'color': 'black',
                        'font-family':'Trebuchet MS',
                        'font-weight':'bold',
                        'text-align':'center',
                        'width':'119.5px'
                           })
    #s = s.set_caption('<h3>Top Worst Hit State</h3><br>')
    pd.options.display.float_format = '{:.2f}'.format
    pd.option_context('display.colheader_justify','center')
    return s;

#display(HTML('<br><br><div style = "font-family:Trebuchet MS;font-size:24px;color:#999999;">Enter the Number to View India Top Worst Hit States</div><br>'))
interact(show_latest_cases, Number='10');


interactive(children=(Text(value='10', description='Number'), Output()), _dom_classes=('widget-interact',))

In [56]:
sorted_statewise_df = statewise_df.sort_values('confirmed',ascending=False)

In [57]:
ipywLayout = widgets.Layout(border='solid 2px green')

def bubble_chart(Scroll):

        figBubble = px.scatter(
                                sorted_statewise_df.head(Scroll),
                                x = 'state', 
                                y ='confirmed',
                                size= 'confirmed',
                                color = 'state', 
                                hover_name='state' ,
                                size_max=60
                              )
        figBubble.update_layout(

                                title="Top Worst Hit States in India",
                                yaxis_title="Confirmed Recoverd & Death Cases",
                                xaxis_title="States",
                                font=dict(family="Trebuchet MS",size=16,color="#778899"),
                                xaxis_showgrid=True,
                                yaxis_showgrid=True,
                                #paper_bgcolor='rgba(0,0,0,0)',
                                plot_bgcolor='rgba(0,0,0,0)',
                                #margin=dict(l=100, r=20, t=0, b=20)

                            )

        #figBubble.update_layout(autosize=False,width=700,height=600),
        #figBubble.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#DCDCDC')
        #figBubble.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#DCDCDC')
        figBubble.update_xaxes(zeroline=True, zerolinewidth=2, zerolinecolor='#DCDCDC')
        figBubble.update_yaxes(zeroline=True, zerolinewidth=2, zerolinecolor='#DCDCDC')

        figBubble.show();

#display(HTML('<br><br><div style = "font-family:Trebuchet MS;font-size:30px;color:#999999;">Scroll to Compare State Top Worst Hit States</div><br>'))

interact(bubble_chart, Scroll=10);

interactive(children=(IntSlider(value=10, description='Scroll', max=30, min=-10), Output()), _dom_classes=('wi…

In [58]:

menu = np.array(statewise_df['state'].values.tolist()) 
menu = np.unique(menu)

StateBarWidget = widgets.Dropdown(options=menu,
    value='Maharashtra',
    description='<div style = "height:100px;width:70px;border-style:#;padding-bottom:10px;font-family:Trebuchet MS;font-size:15px;font-style:bold;font-color:#999999;"><b>States</b></div> :</br></br>',
)

display(HTML('<br><br><br><br>'))
def StateBar(States):
    state = statewise_df.loc[statewise_df['state'] == States]
   
    figBar = go.Figure(
        data=[
            go.Bar(
                name="Confirmed",
                x= state['state'],
                y= state['confirmed'],
                marker=dict(color='#0000FF')
               
                
            ),
            go.Bar(
                name="Deaths",
                x=state['state'],
                y=state['deaths'],
                marker=dict(color='#FF0000')
                
            ),
            go.Bar(
                name="Recovered",
                x=state['state'],
                y=state['recovered'],
                marker=dict(color='#7CFC00')
                
            ),
        
        ],
        layout=go.Layout(
            title="Confirmed Recoverd And Death Cases",
            yaxis_title="Count of Confirmed Recoverd & Death Cases",
            xaxis_title="State",
            font=dict(family="Trebuchet MS",size=16,color="#778899"),
            xaxis_showgrid=True,
            yaxis_showgrid=True,
            width=700,
            height=600,
            paper_bgcolor='rgba(0,0,0,0)',
            plot_bgcolor='rgba(0,0,0,0)'
        )
        
    )
    #figBar.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#DCDCDC')
    #figBar.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#DCDCDC')
    figBar.update_xaxes(zeroline=True, zerolinewidth=2, zerolinecolor='#DCDCDC')
    figBar.update_yaxes(zeroline=True, zerolinewidth=2, zerolinecolor='#DCDCDC'),
    fig.update_yaxes(automargin=True)
    
   
    figBar.show();


#display(HTML('<br><br><div style = "font-family:Trebuchet MS;font-size:24px;color:#999999;">Select the State from Drop Down</div><br>'))    

StateBarWidget.layout.height = '40px' 
interact(StateBar,States = StateBarWidget);

#display(StateBarWidget)


interactive(children=(Dropdown(description='<div style = "height:100px;width:70px;border-style:#;padding-botto…

In [59]:
    
menu = np.array(district_datewise_df['state'].values.tolist()) 
menu = np.unique(menu)


MixedCasesDropdown = widgets.Dropdown(options=menu,
    value='Maharashtra',
    description='<div style = "height:100px;width:70px;border-style:#;padding-bottom:10px;font-family:Trebuchet MS;font-size:15px;font-style:bold;font-color:#999999;"><b>States</b></div> :</br></br>',
    disabled=False
)


def mixedCases(state):
    
    state_df = district_datewise_df.loc[district_datewise_df['state'] == state]
    

    
    date_df = pd.DataFrame((state_df.groupby( [ "date", "current status"] ).sum()).reset_index())
    date_df['date'] = pd.to_datetime(date_df['date'], format='%d/%m/%Y')
    date_df = date_df.sort_values(by=['date'], ascending=True)
    
    
    max_range = date_df['count'].cumsum()
    max_range = max(max_range)
    #print(max_range)
    

    confirmed_df = date_df.loc[date_df['current status'] == 'Hospitalized']
    confirmed_date = confirmed_df['date']
    confirmed_count = confirmed_df['count'].cumsum()
    
    recovered_df = date_df.loc[date_df['current status'] == 'Recovered']
    recovered_count = recovered_df['count'].cumsum()
    recovered_date = recovered_df['date']

    
    deceased_df = date_df.loc[date_df['current status'] == 'Deceased']
    deceased_count = deceased_df['count'].cumsum()
    deceased_date = deceased_df['date']


    
    figLine = go.Figure(
        data=[
            go.Line(
                name='Confirmed',
                x= confirmed_date,
                y= confirmed_count,
                marker=dict(color='#0000FF')
            ),
            
             go.Bar(
                name='Recovered',
                x= recovered_date,
                y= recovered_count,
                marker=dict(color='#7CFC00')
            ),
            
            go.Bar(
                name="Deaths",
                x= deceased_date,
                y= deceased_count,
                marker=dict(color='#FF0000')
            ),
            
    
        ],
    
    
        layout=go.Layout(
            showlegend=True,
            title=state+" Per Day Confirmed Recoverd and Death Cases",
            yaxis_title="Confirmed Recoverd and Death Cases",
            xaxis_title="Dates",
            font=dict(family="Trebuchet MS",size=16,color="#778899"),
            xaxis_showgrid=True,
            yaxis_showgrid=True,
            #paper_bgcolor='rgba(0,0,0,0)',
            plot_bgcolor='rgba(0,0,0,0)'
        )
    )
    
  
    figLine.update_xaxes(rangeslider_visible=True)
    
    figLine.update_yaxes(range=[0, max_range])
    figLine.update_layout(autosize=False,width=975,height=680),
    
    
    #figLine.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#DCDCDC')
    #figLine.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#DCDCDC')
    figLine.update_xaxes(zeroline=True, zerolinewidth=2, zerolinecolor='#DCDCDC')
    figLine.update_yaxes(zeroline=True, zerolinewidth=2, zerolinecolor='#DCDCDC')
    
    display(HTML('<br><br><br><br>'))
    figLine.show();



#display(HTML('<br><br><div style = "font-family:Trebuchet MS;font-size:24px;color:#999999;">Select the State from Drop Down</div><br>'))    
MixedCasesDropdown.layout.height = '40px' 
widgets.interact(mixedCases,state = MixedCasesDropdown);

interactive(children=(Dropdown(description='<div style = "height:100px;width:70px;border-style:#;padding-botto…

In [60]:
menu = np.array(district_datewise_df['state'].values.tolist()) 
menu = np.unique(menu)

ConfirmedDropdown = widgets.Dropdown(options=menu,
        value='Maharashtra',
        description='<div style = "height:100px;width:70px;border-style:#;padding-bottom:10px;font-family:Trebuchet MS;font-size:15px;font-style:bold;font-color:#999999;"><b>States</b></div> :</br></br>',

    )


def ConfirmedDateWiseCases(state):
    
    state_df = district_datewise_df.loc[district_datewise_df['state'] == state]
    

    
    date_df = pd.DataFrame((state_df.groupby( [ "date", "current status"] ).sum()).reset_index())
    date_df['date'] = pd.to_datetime(date_df['date'], format='%d/%m/%Y')
    date_df = date_df.sort_values(by=['date'], ascending=True)
    
    
    max_range = date_df['count'].cumsum()
    max_range = max(max_range)
   

    confirmed_df = date_df.loc[date_df['current status'] == 'Hospitalized']
    confirmed_date = confirmed_df['date']
    confirmed_count = confirmed_df['count'].cumsum()
    
    
    
    figLine = go.Figure(
        data=[
        
            go.Line(
                name="Confirmed",
                x= confirmed_date,
                y= confirmed_count,
                line={'color': '#0000FF'}

            
            ),

        ],
        layout=go.Layout(
            showlegend=True,
            title=state+" Confirmed Cases" ,
            yaxis_title="Confirmed Cases",
            xaxis_title="Dates",
            font=dict(family="Trebuchet MS",size=16,color="#778899"),
            xaxis_showgrid=True,
            yaxis_showgrid=True,
            #paper_bgcolor='rgba(0,0,0,0)',
            plot_bgcolor='rgba(0,0,0,0)'
        )
    )
    figLine.update_xaxes(rangeslider_visible=True)
    figLine.update_yaxes(range=[0, max_range])
    figLine.update_layout(autosize=False,width=975,height=680)
    
    
    #figLine.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#DCDCDC')
    #fbbigLine.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#DCDCDC')
    figLine.update_xaxes(zeroline=True, zerolinewidth=2, zerolinecolor='#DCDCDC')
    figLine.update_yaxes(zeroline=True, zerolinewidth=2, zerolinecolor='#DCDCDC')
    
    display(HTML('<br><br><br><br>'))
    figLine.show();

    

#display(HTML('<br><br><div style = "font-family:Trebuchet MS;font-size:24px;color:#999999;">Select the State from Drop Down</div><br>'))    
ConfirmedDropdown.layout.height = '40px' 
widgets.interact(ConfirmedDateWiseCases,state = ConfirmedDropdown);

interactive(children=(Dropdown(description='<div style = "height:100px;width:70px;border-style:#;padding-botto…

In [61]:
menu = np.array(district_datewise_df['state'].values.tolist()) 
menu = np.unique(menu)

RecoveredDropdown = widgets.Dropdown(options=menu,
    value='Maharashtra',
    description='<div style = "height:100px;width:70px;border-style:#;padding-bottom:10px;font-family:Trebuchet MS;font-size:15px;font-style:bold;font-color:#999999;"><b>States</b></div> :</br></br>',
    disabled=False
)                                                                       

def recoverdDateWise(state):
    
    
    
    state_df = district_datewise_df.loc[district_datewise_df['state'] == state]
    

    
    date_df = pd.DataFrame((state_df.groupby( [ "date", "current status"] ).sum()).reset_index())
    date_df['date'] = pd.to_datetime(date_df['date'], format='%d/%m/%Y')
    date_df = date_df.sort_values(by=['date'], ascending=True)


    recovered_df = date_df.loc[date_df['current status'] == 'Recovered']
    recovered_count = recovered_df['count'].cumsum()
    recovered_date = recovered_df['date']
    max_range = max(recovered_count)

    
    figLine = go.Figure(
        data=[
            go.Scatter(
                name="Recovered",
                x= recovered_date,
                y= recovered_count,
                line={'color': '#7CFC00'},
                
            ),
    ],
    layout=go.Layout(
            showlegend=True,
            title=state+" Recoverd Cases",
            yaxis_title="Recovered Cases",
            xaxis_title="Dates",
            font=dict(family="Trebuchet MS",size=16,color="#778899"),
            xaxis_showgrid=True,
            yaxis_showgrid=True,
            #paper_bgcolor='rgba(0,0,0,0)',
            plot_bgcolor='rgba(0,0,0,0)'
            
        
        )
    
    )
    figLine.update_xaxes(rangeslider_visible=True),
    figLine.update_yaxes(range=[0, max_range])
    figLine.update_layout(autosize=False,width=975,height=680)
    
    
    #figLine.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#DCDCDC')
    #figLine.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#DCDCDC')
    figLine.update_xaxes(zeroline=True, zerolinewidth=2, zerolinecolor='#DCDCDC')
    figLine.update_yaxes(zeroline=True, zerolinewidth=2, zerolinecolor='#DCDCDC')
    display(HTML('<br><br><br><br>'))
    figLine.show();

    
    


#display(HTML('<br><br><div style = "font-family:Trebuchet MS;font-size:24px;color:#999999;">Select the State from Drop Down</div><br>'))    
RecoveredDropdown.layout.height = '40px' 
widgets.interact(recoverdDateWise,state = RecoveredDropdown);

interactive(children=(Dropdown(description='<div style = "height:100px;width:70px;border-style:#;padding-botto…

In [62]:

menu = np.array(district_datewise_df['state'].values.tolist()) 
menu = np.unique(menu)
 

DeathDropdown = widgets.Dropdown(options=menu,
    value='Maharashtra',
    description='<div style = "height:100px;width:70px;border-style:#;padding-bottom:10px;font-family:Trebuchet MS;font-size:15px;font-style:bold;font-color:#999999;"><b>States</b></div> :</br></br>',

)


def DeathDateWise(state):
    
    state_df = district_datewise_df.loc[district_datewise_df['state'] == state]
    

    
    date_df = pd.DataFrame((state_df.groupby( [ "date", "current status"] ).sum()).reset_index())
    date_df['date'] = pd.to_datetime(date_df['date'], format='%d/%m/%Y')
    date_df = date_df.sort_values(by=['date'], ascending=True)
    

    deceased_df = date_df.loc[date_df['current status'] == 'Deceased']
    deceased_count = deceased_df['count'].cumsum()
    deceased_date = deceased_df['date']
    max_range_death = max(deceased_count)
    
    figLine = go.Figure(
        data=[
        
            go.Line(
                name="Deaths",
                x= deceased_date,
                y= deceased_count,
                line={'color': '#FF0000'}

            ),
        
        ],
        layout=go.Layout(
            showlegend=True,
            title=state+" Deaths Cases",
            yaxis_title="Deaths Cases",
            xaxis_title="Dates",
            font=dict(family="Trebuchet MS",size=16,color="#778899"),
            xaxis_showgrid=True,
            yaxis_showgrid=True,
            #paper_bgcolor='rgba(0,0,0,0)',
            plot_bgcolor='rgba(0,0,0,0)'
        )
    )
    figLine.update_xaxes(rangeslider_visible=True)
    figLine.update_yaxes(range=[0, max_range_death])
    figLine.update_layout(autosize=False,width=975,height=680)
    
    
    #figLine.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#DCDCDC')
    #figLine.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#DCDCDC')
    figLine.update_xaxes(zeroline=True, zerolinewidth=2, zerolinecolor='#DCDCDC')
    figLine.update_yaxes(zeroline=True, zerolinewidth=2, zerolinecolor='#DCDCDC')
    display(HTML('<br><br><br><br>'))
    figLine.show();



#display(HTML('<br><br><div style = "font-family:Trebuchet MS;font-size:24px;color:#999999;">Select the State from Drop Down</div><br>'))    
DeathDropdown.layout.height = '40px' 
widgets.interact(DeathDateWise,state = DeathDropdown);

interactive(children=(Dropdown(description='<div style = "height:100px;width:70px;border-style:#;padding-botto…

In [63]:
#display(HTML('<br><br><div style = "font-family:Trebuchet MS;font-size:24px;color:#999999;">Hover Over Map</div><br>'))    


northing = 20.5937
easting = 78.9629
india_map = folium.Map(location=[northing,easting], tiles="cartodbpositron", zoom_start=4, max_zoom = 10, min_zoom = 2)
            

for i in range(0,len(lat_long_merged_df)):
    
    confirmed =  lat_long_merged_df.iloc[i,1]
    confirmed =  str(confirmed)
    deaths =     lat_long_merged_df.iloc[i,3]
    deaths =      str(deaths)
    
    recovered =     lat_long_merged_df.iloc[i,2]
    recovered =      str(recovered)
    
    active =     lat_long_merged_df.iloc[i,4]
    active =      str(active)
    
    death_rate=  np.round(lat_long_merged_df.iloc[i,3]/(lat_long_merged_df.iloc[i,3]+1.00001)*100,2)
    death_rate =      str(death_rate)
    
    
    folium.Marker(
        location=[ lat_long_merged_df.iloc[i]['lat'], lat_long_merged_df.iloc[i]['long'] ],
        fill=True,
        #radius=((np.log(lat_long_merged_df.iloc[i,2]+1.00001))+0.2)*50000,
        color=' #ffb3d9',
        fill_color='white',
        tooltip = "<div style='margin: 0; background-color: #cccccc; color: #000000;'>"+
                    "<h4 style='text-align:center;font-weight: bold'>"+lat_long_merged_df.iloc[i]['State'] + "</h4>"
                    "<hr style='margin:10px;color: #4d4d4d;'>"+
                    "<ul style='color: #000000;font-weight: bold;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
                        "<li>Confirmed: "+confirmed+"</li>"+
                        "<li>Deaths:   "+deaths+"</li>"+
                         "<li>Recovered: "+ recovered + "</li>"+
                         "<li>Active: "+ active + "</li>"+
                        "<li>Death Rate: "+death_rate + "</li>"+
                    "</ul></div>"
    ).add_to(india_map)

india_map



In [64]:
display(HTML(
    
            '<html>'
                '<head>'
                         '<style>'
                                'a:link {color: #0000ff;}'              # unvisited link
                                'a:visited {color: #e600ac;}'           # visited link 
                                'a:hover {color: #2eb82e;}'            # mouse over link
                                'a:active {color: #00ff00;}'          # selected link 
                       '</style>'
                    '</head>'
                '<body>'

                    '<div style = "Lucida Console, Courier, monospace;font-size:20px;color:#778899;line-height: 1.7;text-align: justify;">'+

                        '<h2>Notebook covers:</h2>'
                         
                            '1. What is COVID-19.<br>'+
                            '2. Data loading from COVID19-India API repository.<br>'+
                            '3. Data Cleaning and Preparation.<br>'+
                            '4. Visualising N number of worst hit State using plotly scatter plot.<br>'+
                            '5. Plotting confirmed, recovered and death cases for the requested State.<br>'+
                            '6. Plotting all cases on State map using Folium.</li><br>'+
                        

                        '<h2>Symptoms:</h2>'+
                        'People may be sick with the virus for 1 to 14 days before developing symptoms. The most common symptoms of coronavirus disease (COVID-19) are fever, tiredness, and dry cough. Most people (about 80%)  recover from the disease without needing special treatment.<br>'+
                          
                            '1. Cough.<br>'+
                            '2. Fever.<br>'+
                            '3. Tiredness.<br>'+
                            '4. Difficulty in breathing (severe cases).'+
                            '<p><img src="symprre.jpg" align="center"></img></p>'+
                        
                '</div>'
          '</body>'
        '</html>'
  

        ))
    


In [65]:
%%HTML
<div style = "Lucida Console, Courier, monospace;font-size:20px;color:#778899;line-height: 1.7;text-align: justify;">
<h2>More Info on COVID-19:</h2>
<a style="text-decoration:none" href = https://www.who.int/health-topics/coronavirus>  https://www.who.int/health-topics/coronavirus </a><br>
<a style="text-decoration:none" href = https://www.who.int/emergencies/diseases/novel-coronavirus-2019>  https://www.who.int/emergencies/diseases/novel-coronavirus-2019  </a><br>
<a style="text-decoration:none" href = https://www.nature.com/articles/s41597-020-0448-0>  https://www.nature.com/articles/s41597-020-0448-0  </a> 
</div>
<div style = "Lucida Console, Courier, monospace;font-size:20px;color:#778899;">
<marquee direction="left" height="100px" width="90%" scrollamount="15"><h2>Disclaimer: This is not a official website. For more Details Visit to the Given Link  !!</h2></marquee>
</div>
    

